# 과거 미국 주식 랠리 분석 — 다음 랠리의 승자는 누구인가?

**분석 기간: 2026.02.02 ~**

> **Executive Summary**
> 
> 1988~2000년 닷컴 버블을 **12년 전체(그룹 1)**와 **버블 말기 1년(그룹 2)**으로 나눠 분석한 결과:
> 
> **그룹 1 (162개 종목, 12년)**: 중앙값 3.88배 수익, 손실 비율 2%. 슈퍼 성장 클러스터(7.4배)에서 Financial Services(14개)가 Technology(7개)보다 많았다. 12년간 일관되게 나스닥을 이긴 종목은 단 3개(AB, AEG, LEG).
> 
> **그룹 2 (339개 종목, 1년)**: 중앙값 1.08배, 손실 비율 37%. Technology가 폭등 클러스터에 집중되었으나 양극화 극심.
> 
> **주요 한계**: 현재 상장 종목 기준으로 수집하여 **Survivorship Bias 존재**. 상장폐지된 닷컴 버블 핵심 종목(YHOO, SUNW 등)이 누락되어 손실 비율이 과소추정되었을 가능성이 높다.

---

## 배경

2026년 하반기, 중간선거를 앞두고 미국 주식시장의 랠리가 예상된다. 이후 AI 버블이 본격화될 가능성까지 고려하면, 지금 시점에서 **"랠리 때 어떤 종목이 주로 상승하는가"** 를 과거 데이터로 검증해볼 필요가 있다.

분석 대상으로 **1988~2000년 닷컴 버블 랠리**를 선택했다. 기술 혁신이 주도한 장기 상승장이라는 점에서 현재 AI 랠리와 구조적 유사성이 있기 때문이다.

## 질문 설정

1. 12년간의 랠리에서 실제로 큰 수익을 낸 종목은 어떤 특징을 갖고 있는가?
2. 테크 섹터만 올랐는가, 아니면 다른 섹터에서도 승자가 나왔는가?
3. 버블 말기(1년)와 전체 랠리(12년)의 패턴은 어떻게 다른가?
4. 이 패턴이 현재 AI 랠리에 시사하는 점은?

## 분석 대상

| 구분 | 기간 | 설명 |
|------|------|------|
| 그룹 1 (Full Coverage) | 1988.10 ~ 2000.03 | 12년 전체 랠리를 경험한 장기 생존 종목 162개 |
| 그룹 2 (Bubble End) | 1999.03 ~ 2000.03 | 버블 말기 1년 동안 존재한 전체 종목 339개 |

## 한계

- **Survivorship Bias**: 현재 상장 종목 기준으로 수집. 상장폐지 종목(YHOO, SUNW, CMGI 등)이 누락되어 손실 비율과 하락 클러스터가 과소추정됨
- 이로 인해 "12년 보유 시 손실 2%"라는 결과는 실제보다 낙관적일 가능성 높음
- 추후 FMP API로 상장폐지 종목까지 포함하여 재분석 예정

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings('ignore')

# 분석 기간 설정
START_DATE = "1988-10-01"
END_DATE = "2000-03-31"

print(f"분석 기간: {START_DATE} ~ {END_DATE}")

## 1. 데이터 수집

NASDAQ Trader에서 현재 상장된 전체 종목(NASDAQ + NYSE 등) 티커를 수집하고, 주요 지수 심볼을 추가한다.

In [ ]:
all_tickers = set()

# --- NASDAQ Trader FTP - 전체 상장 종목 ---
try:
    # NASDAQ 거래소 상장 종목
    nasdaq_listed = pd.read_csv(
        "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt",
        sep="|"
    )
    nasdaq_listed = nasdaq_listed[nasdaq_listed["Test Issue"] == "N"]
    nasdaq_syms = set(nasdaq_listed["Symbol"].dropna().tolist())
    # 기타 거래소 (NYSE 등)
    other_listed = pd.read_csv(
        "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt",
        sep="|"
    )
    if "Test Issue" in other_listed.columns:
        other_listed = other_listed[other_listed["Test Issue"] == "N"]
    if "ACT Symbol" in other_listed.columns:
        other_syms = set(other_listed["ACT Symbol"].dropna().tolist())
    elif "Symbol" in other_listed.columns:
        other_syms = set(other_listed["Symbol"].dropna().tolist())
    else:
        other_syms = set()
    
    exchange_tickers = nasdaq_syms | other_syms
    # 일반 주식만 필터링 (ETF, 워런트 등 제외를 위해 특수문자 포함 티커 제거)
    exchange_tickers = {t for t in exchange_tickers if isinstance(t, str) and t.isalpha() and len(t) <= 5}
    all_tickers.update(exchange_tickers)
    print(f"NASDAQ Trader (NASDAQ + 기타 거래소): {len(exchange_tickers)}개")
except Exception as e:
    print(f"NASDAQ Trader 수집 실패: {e}")

# --- 주요 지수 심볼 추가 ---
index_tickers = {"^GSPC", "^DJI", "^IXIC", "^RUT", "^NYA"}
all_tickers.update(index_tickers)

print(f"\n총 고유 티커 수: {len(all_tickers)}개")

## 2. yfinance 배치 다운로드

11,724개 티커를 100개씩 배치로 나눠 1988.10~2000.03 기간의 종가 데이터를 다운로드한다. 현재 상장 종목 기준이므로 해당 기간에 존재하지 않았던 종목은 데이터가 없다.

In [ ]:
def download_batch(tickers, start, end, batch_size=100):
    """티커를 배치로 나눠서 yfinance에서 다운로드"""
    ticker_list = sorted(list(tickers))
    all_data = {}
    failed = []
    
    total_batches = (len(ticker_list) + batch_size - 1) // batch_size
    
    for i in range(0, len(ticker_list), batch_size):
        batch = ticker_list[i:i+batch_size]
        batch_num = i // batch_size + 1
        print(f"배치 {batch_num}/{total_batches} 다운로드 중... ({len(batch)}개 티커)")
        
        try:
            data = yf.download(
                tickers=batch,
                start=start,
                end=end,
                auto_adjust=True,
                progress=False,
                threads=True
            )
            
            if data.empty:
                continue
            
            # 단일 티커인 경우 MultiIndex가 아님
            if len(batch) == 1:
                ticker = batch[0]
                if not data.empty:
                    all_data[ticker] = data["Close"]
            else:
                # MultiIndex columns: (Price, Ticker)
                if isinstance(data.columns, pd.MultiIndex):
                    close_data = data["Close"]
                else:
                    close_data = data
                
                for ticker in close_data.columns:
                    col = close_data[ticker].dropna()
                    if len(col) > 0:
                        all_data[ticker] = col
                        
        except Exception as e:
            failed.extend(batch)
            print(f"  배치 {batch_num} 실패: {e}")
        
        # API rate limit 방지
        if batch_num % 5 == 0:
            time.sleep(1)
    
    print(f"\n다운로드 완료: {len(all_data)}개 종목 데이터 확보")
    if failed:
        print(f"실패한 티커: {len(failed)}개")
    
    return all_data

# 다운로드 실행
stock_data = download_batch(all_tickers, START_DATE, END_DATE, batch_size=100)

## 3. 데이터 필터링

11,724개 중 360개만 해당 기간 데이터가 존재했다. 대부분의 현재 상장 종목은 2000년 이후 IPO이므로 정상적인 결과다. 최소 250거래일(약 1년) 이상 데이터를 보유한 339개 종목을 분석 대상으로 선정한다.

In [ ]:
# 최소 거래일 수 기준으로 필터링 (최소 250일 = 약 1년)
MIN_TRADING_DAYS = 250

valid_tickers = {k: v for k, v in stock_data.items() if len(v) >= MIN_TRADING_DAYS}

print(f"전체 다운로드 종목: {len(stock_data)}개")
print(f"최소 {MIN_TRADING_DAYS}거래일 이상 데이터 보유: {len(valid_tickers)}개")

# 종합 DataFrame 생성
close_df = pd.DataFrame(valid_tickers)
close_df.index = pd.to_datetime(close_df.index)
close_df = close_df.sort_index()

print(f"\n데이터 기간: {close_df.index[0].date()} ~ {close_df.index[-1].date()}")
print(f"총 거래일: {len(close_df)}")
print(f"종목 수: {close_df.shape[1]}")

# 기간별 커버리지 확인
coverage = close_df.notna().sum(axis=1)
print(f"\n일자별 평균 종목 수: {coverage.mean():.0f}")
print(f"일자별 최소 종목 수: {coverage.min()} ({coverage.idxmin().date()})")
print(f"일자별 최대 종목 수: {coverage.max()} ({coverage.idxmax().date()})")

# 수익률 계산
returns_df = close_df.pct_change()

# 기간 전체 수익률 (처음~끝)
total_returns = {}
for col in close_df.columns:
    series = close_df[col].dropna()
    if len(series) >= 2:
        total_returns[col] = (series.iloc[-1] / series.iloc[0] - 1) * 100

total_returns_series = pd.Series(total_returns).sort_values(ascending=False)

print(f"\n=== 기간 전체 수익률 상위 20 ===")
print(total_returns_series.head(20).to_string(float_format="{:.1f}%".format))
print(f"\n=== 기간 전체 수익률 하위 10 ===")
print(total_returns_series.tail(10).to_string(float_format="{:.1f}%".format))

In [ ]:
# 결측치 처리: 기간별 분리
# 첫 번째 분석 대상: 1988-10-01 ~ 2000-03-31 기간에 결측치가 없는 데이터: 162개 종목
close_df_1 = close_df.loc["1988-10-01":"2000-03-31"]
close_df_1 = close_df_1.dropna(axis=1)
print(f"첫 번째 분석 대상 종목 수: {close_df_1.shape[1]}개")

# 두 번째 분석 대상: 1999-03-31 ~ 2000-12-31 기간에 결측치가 없는 데이터: 339개 종목
close_df_2 = close_df.loc["1999-03-31":"2000-12-31"]
close_df_2 = close_df_2.dropna(axis=1) # 만약을 대비
print(f"두 번째 분석 대상 종목 수: {close_df_2.shape[1]}개")

In [ ]:
# 정규화 (시작점 = 100)
normalized_1 = close_df_1.apply(lambda x: x / x.iloc[0] * 100)
print(f"그룹 1 정규화 완료: {normalized_1.shape}")

## 4. 시각화 — 162개 풀 커버리지 종목의 랠리 흐름

339개 종목 중 12년 전체 기간에 결측치가 없는 162개 종목(그룹 1)을 시작가=100으로 정규화하여 시각화한다. 개별 종목은 배경으로, 전체 중앙값을 기준선으로 표시한다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from collections import defaultdict, Counter

# ==================== 공통 함수 정의 ====================

def collect_sectors(tickers):
    """yfinance에서 섹터 정보 수집"""
    sectors = {}
    for ticker in tickers:
        try:
            info = yf.Ticker(ticker).info
            sectors[ticker] = info.get('sector', 'Unknown')
        except:
            sectors[ticker] = 'Unknown'
    return pd.Series(sectors)

def create_sector_colors(sector_series):
    """섹터별 색상 매핑 생성"""
    unique_sectors = sector_series.unique()
    cmap = plt.cm.get_cmap('tab10', len(unique_sectors))
    return {sector: cmap(i) for i, sector in enumerate(sorted(unique_sectors))}

def plot_sector_chart(normalized_df, sector_series, sector_colors, title):
    """섹터별 색상으로 로그 스케일 차트 그리기"""
    fig, ax = plt.subplots(figsize=(14, 7))
    ax.set_yscale('log')
    
    for sector, tickers in sector_series.groupby(sector_series):
        color = sector_colors.get(sector, 'grey')
        for ticker in tickers.index:
            if ticker in normalized_df.columns:
                ax.plot(normalized_df[ticker], color=color, alpha=0.3, linewidth=0.7)
        ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({len(tickers)})')
    
    ax.plot(normalized_df.median(axis=1), color='black', linewidth=2.5, linestyle='--', label='Median')
    ax.legend(fontsize=8, loc='upper left', ncol=2)
    ax.set_ylabel('Normalized Price (Start=100, Log Scale)')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

def find_optimal_clusters(X, k_range=range(2, 11)):
    """Elbow Method와 Silhouette Score로 최적 클러스터 수 탐색"""
    inertias, sil_scores = [], []
    for k in k_range:
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = km.fit_predict(X)
        inertias.append(km.inertia_)
        sil_scores.append(silhouette_score(X, labels))
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].plot(k_range, inertias, marker='o')
    axes[0].set_title('Elbow Method')
    axes[0].set_xlabel('k')
    axes[0].set_ylabel('Inertia')
    axes[1].plot(k_range, sil_scores, marker='o', color='orange')
    axes[1].set_title('Silhouette Score')
    axes[1].set_xlabel('k')
    axes[1].set_ylabel('Score')
    plt.tight_layout()
    plt.show()
    return inertias, sil_scores

def run_clustering(normalized_df, n_clusters):
    """K-Means 클러스터링 실행"""
    log_norm = np.log(normalized_df)
    X = StandardScaler().fit_transform(log_norm.T)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    cluster_df = pd.DataFrame({'Ticker': normalized_df.columns, 'Cluster': clusters})
    return kmeans, cluster_df, X

def plot_cluster_chart(normalized_df, cluster_df, sector_series, sector_colors, n_clusters):
    """클러스터별 섹터 색상 차트 그리기"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()
    
    for c in range(n_clusters):
        ax = axes[c]
        ax.set_yscale('log')
        
        cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
        cluster_data = normalized_df[cluster_tickers]
        
        for ticker in cluster_tickers:
            sector = sector_series.get(ticker, 'Unknown')
            color = sector_colors.get(sector, 'grey')
            ax.plot(cluster_data[ticker], color=color, alpha=0.3, linewidth=0.7)
        
        cluster_median = cluster_data.median(axis=1)
        ax.plot(cluster_median, color='black', linewidth=2.5, linestyle='--', label=f'Cluster {c} Median')
        
        cluster_sectors = sector_series[cluster_tickers].value_counts()
        for sector, count in cluster_sectors.items():
            ax.plot([], [], color=sector_colors.get(sector, 'grey'), linewidth=2, label=f'{sector} ({count})')
        
        ax.legend(fontsize=7, loc='upper left', ncol=2)
        ax.set_title(f'Cluster {c} ({len(cluster_tickers)} stocks)')
        ax.set_ylabel('Normalized Price (Log)')
    
    # 남는 subplot 숨기기
    for i in range(n_clusters, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def summarize_clusters(normalized_df, cluster_df, sector_series, n_clusters):
    """클러스터별 통계 요약 출력"""
    for c in range(n_clusters):
        cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
        cluster_data = normalized_df[cluster_tickers]
        final_vals = cluster_data.iloc[-1]
        
        print(f"=== Cluster {c} ({len(cluster_tickers)} stocks) ===")
        print(f"  중앙값 최종: {cluster_data.median(axis=1).iloc[-1]:.1f} (={cluster_data.median(axis=1).iloc[-1]/100:.1f}x)")
        print(f"  평균 최종:   {final_vals.mean():.1f} (={final_vals.mean()/100:.1f}x)")
        print(f"  범위:        {final_vals.min():.1f} ~ {final_vals.max():.1f}")
        
        cluster_sectors = sector_series[cluster_tickers].value_counts()
        print(f"  섹터 구성:")
        for sector, count in cluster_sectors.items():
            print(f"    {sector}: {count}개")
        
        top3 = final_vals.nlargest(3)
        print(f"  상위 3종목: {', '.join([f'{t}({v:.0f})' for t, v in top3.items()])}")
        print()

def calculate_rs(normalized_df, index_ticker='^IXIC'):
    """상대강도(RS) 계산"""
    if index_ticker not in normalized_df.columns:
        print(f"{index_ticker} 데이터가 없습니다")
        return None
    index_norm = normalized_df[index_ticker] / normalized_df[index_ticker].iloc[0]
    rs_df = normalized_df.div(index_norm * 100, axis=0)
    return rs_df

def classify_rs_pattern(rs_series, median_series):
    """RS 패턴을 4가지로 분류"""
    rs = rs_series.dropna()
    med = median_series.loc[rs.index]
    
    if len(rs) < 10:
        return None
    
    mid = len(rs) // 2
    first_half, second_half = rs.iloc[:mid], rs.iloc[mid:]
    med_first, med_second = med.iloc[:mid], med.iloc[mid:]
    
    first_above = (first_half > med_first).mean() > 0.6
    first_below = (first_half < med_first).mean() > 0.6
    second_above = (second_half > med_second).mean() > 0.6
    second_below = (second_half < med_second).mean() > 0.6
    
    if first_above and second_above: return 1  # 항상 위
    elif first_below and second_above: return 2  # 골든크로스
    elif first_above and second_below: return 3  # 데드크로스
    elif first_below and second_below: return 4  # 항상 아래
    else: return 0  # 분류 불가

def analyze_rs_patterns(rs_df, periods, sector_series, index_ticker='^IXIC'):
    """기간별 RS 패턴 분석"""
    pattern_results = {}
    
    for period_name, start, end in periods:
        rs_period = rs_df.loc[start:end]
        rs_period_norm = rs_period / rs_period.iloc[0]
        rs_median = rs_period_norm.drop(columns=[index_ticker], errors='ignore').median(axis=1)
        
        period_patterns = {}
        for ticker in rs_period_norm.columns:
            if ticker == index_ticker:
                continue
            pattern = classify_rs_pattern(rs_period_norm[ticker], rs_median)
            if pattern is not None:
                period_patterns[ticker] = pattern
        
        pattern_results[period_name] = period_patterns
        
        counts = Counter(period_patterns.values())
        print(f"\n=== {period_name} ===")
        print(f"1. 항상 위: {counts.get(1, 0)}개")
        print(f"2. 골든크로스: {counts.get(2, 0)}개")
        print(f"3. 데드크로스: {counts.get(3, 0)}개")
        print(f"4. 항상 아래: {counts.get(4, 0)}개")
        print(f"0. 분류불가: {counts.get(0, 0)}개")
    
    return pattern_results

def find_consistent_patterns(pattern_results, sector_series):
    """전 기간 일관된 패턴 종목 찾기"""
    ticker_patterns = defaultdict(list)
    for period_name, patterns in pattern_results.items():
        for ticker, pattern in patterns.items():
            ticker_patterns[ticker].append((period_name, pattern))
    
    n_periods = len(pattern_results)
    always_above = [t for t, p in ticker_patterns.items() if len(p) == n_periods and all(x[1] == 1 for x in p)]
    always_below = [t for t, p in ticker_patterns.items() if len(p) == n_periods and all(x[1] == 4 for x in p)]
    rising = [t for t, p in ticker_patterns.items() if len(p) == n_periods and p[0][1] in [4, 0] and p[-1][1] == 1]
    falling = [t for t, p in ticker_patterns.items() if len(p) == n_periods and p[0][1] == 1 and p[-1][1] in [4, 0]]
    
    print("=== 전 기간 일관된 패턴 ===\n")
    print(f"1. 항상 나스닥을 이긴 종목 ({len(always_above)}개):")
    for t in always_above:
        print(f"  {t} ({sector_series.get(t, 'Unknown')})")
    print(f"\n2. 항상 나스닥에 진 종목 ({len(always_below)}개):")
    for t in always_below:
        print(f"  {t} ({sector_series.get(t, 'Unknown')})")
    print(f"\n3. 초기 약세 → 후기 강세 ({len(rising)}개):")
    for t in rising:
        print(f"  {t} ({sector_series.get(t, 'Unknown')})")
    print(f"\n4. 초기 강세 → 후기 약세 ({len(falling)}개):")
    for t in falling:
        print(f"  {t} ({sector_series.get(t, 'Unknown')})")
    
    return always_above, always_below, rising, falling

### 로그 스케일 + 섹터별 색상

가격 편차가 100배 이상이므로 로그 스케일을 적용해야 수익률 흐름이 정확히 보인다. 개별 종목에 섹터별 색상을 입혀 **질문 2: "테크 섹터만 올랐는가?"** 를 시각적으로 확인한다.

In [ ]:
# 섹터 정보 수집
sectors = {}
for ticker in close_df_1.columns:
    try:
        info = yf.Ticker(ticker).info
        sectors[ticker] = info.get('sector', 'Unknown')
    except Exception as e:
        sectors[ticker] = 'Unknown'
sector_df = pd.Series(sectors)
sector_df.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
ax.set_yscale('log')

# 섹터별 색상 매핑
unique_sectors = sector_df.unique()
cmap = plt.cm.get_cmap('tab10', len(unique_sectors))
sector_colors = {sector: cmap(i) for i, sector in enumerate(sorted(unique_sectors))}

# 섹터별로 개별 종목 그리기
for sector, tickers in sector_df.groupby(sector_df):
    color = sector_colors[sector]
    for ticker in tickers.index:
        if ticker in normalized_1.columns:
            ax.plot(normalized_1[ticker], color=color, alpha=0.3, linewidth=0.7)
    # 범례용 더미 라인 (섹터당 1개)
    ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({len(tickers)})')

# 중앙값 선
ax.plot(normalized_1.median(axis=1), color='black', linewidth=2.5, linestyle='--', label='Median')

ax.legend(fontsize=8, loc='upper left', ncol=2)
ax.set_ylabel('Normalized Price (Start=100, Log Scale)')
ax.set_title('1988.10 ~ 2000.03 Rally by Sector (162 stocks)')
plt.tight_layout()
plt.show()

## 5. 클러스터링 — 질문 1: "큰 수익을 낸 종목의 특징은?"

로그 정규화된 가격 시계열을 K-Means로 클러스터링하여, 비슷한 시기에 비슷한 주가 흐름을 보인 종목들을 그룹화한다. 각 클러스터의 섹터 구성과 수익률 분포를 비교하여 **슈퍼 성장주의 공통점**을 찾는다.

### K-Means Clustering

Elbow Method와 Silhouette Score로 최적 클러스터 수를 결정한 뒤, 클러스터별 소속 종목을 섹터 색상으로 시각화한다.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
# 적절한 클러스터 수 찾기 Elbow Method, Silhouette Score
from sklearn.metrics import silhouette_score

inertias = []
sil_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X, labels))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Elbow Method
axes[0].plot(K_range, inertias, marker='o')
axes[0].set_title('Elbow Method')
axes[0].set_xlabel('Number of clusters (k)')
axes[0].set_ylabel('Inertia')

# Silhouette Score
axes[1].plot(K_range, sil_scores, marker='o', color='orange')
axes[1].set_title('Silhouette Score')
axes[1].set_xlabel('Number of clusters (k)')
axes[1].set_ylabel('Score')

plt.tight_layout()
plt.show()

In [ ]:
# 최적 n_clusters = 4
log_norm_1 = np.log(normalized_1)
X = StandardScaler().fit_transform(log_norm_1.T)
kmeans = KMeans(n_clusters=4, random_state=42)

clusters = kmeans.fit_predict(X)
cluster_df = pd.DataFrame({'Ticker': normalized_1.columns, 'Cluster': clusters})
cluster_df.head(10)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for c in range(kmeans.n_clusters):
    ax = axes[c]
    ax.set_yscale('log')
    
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    cluster_data = normalized_1[cluster_tickers]
    
    # 개별 종목을 섹터 색상으로 그리기
    for ticker in cluster_tickers:
        sector = sector_df.get(ticker, 'Unknown')
        color = sector_colors.get(sector, 'grey')
        ax.plot(cluster_data[ticker], color=color, alpha=0.3, linewidth=0.7)
    
    # 해당 클러스터의 중앙값 (군집 중앙점)
    cluster_median = cluster_data.median(axis=1)
    ax.plot(cluster_median, color='black', linewidth=2.5, linestyle='--', label=f'Cluster {c} Median')
    
    # 섹터 범례 (이 클러스터에 존재하는 섹터만)
    cluster_sectors = sector_df[cluster_tickers].value_counts()
    for sector, count in cluster_sectors.items():
        ax.plot([], [], color=sector_colors.get(sector, 'grey'), linewidth=2, label=f'{sector} ({count})')
    
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    ax.set_title(f'Cluster {c} ({len(cluster_tickers)} stocks)')
    ax.set_ylabel('Normalized Price (Log)')

plt.tight_layout()
plt.show()

In [ ]:
for c in range(kmeans.n_clusters):
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    cluster_data = normalized_1[cluster_tickers]
    
    # 최종 수익률
    final_vals = cluster_data.iloc[-1]
    
    print(f"=== Cluster {c} ({len(cluster_tickers)} stocks) ===")
    print(f"  중앙값 최종: {cluster_data.median(axis=1).iloc[-1]:.1f} (={cluster_data.median(axis=1).iloc[-1]/100:.1f}x)")
    print(f"  평균 최종:   {final_vals.mean():.1f} (={final_vals.mean()/100:.1f}x)")
    print(f"  범위:        {final_vals.min():.1f} ~ {final_vals.max():.1f}")
    
    # 섹터 구성
    cluster_sectors = sector_df[cluster_tickers].value_counts()
    print(f"  섹터 구성:")
    for sector, count in cluster_sectors.items():
        print(f"    {sector}: {count}개")
    
    # 대표 종목 (최종 정규화 값 기준 상위 3)
    top3 = final_vals.nlargest(3)
    print(f"  상위 3종목: {', '.join([f'{t}({v:.0f})' for t, v in top3.items()])}")
    print()


## 6. 상대강도 분석 — 시장을 이긴 진짜 승자 찾기

종목 가격 / S&P 500 지수 가격으로 상대강도를 계산한다. 이 수치가 우상향하는 구간이 **시장보다 강한 진짜 랠리**이며, 단순히 시장과 함께 오른 종목과 초과수익을 낸 종목을 구분할 수 있다.

In [ ]:
# 상대강도(RS) = 종목 정규화 가격 / NASDAQ 정규화 가격
# 시작점=1로 정규화하여 비교
# 1보다 위 = 나스닥 초과수익
nasdaq = close_df_1['^IXIC'] if '^IXIC' in close_df_1.columns else None

if nasdaq is not None:
    nasdaq_norm = nasdaq / nasdaq.iloc[0]
    rs_df = normalized_1.div(nasdaq_norm * 100, axis=0)  # normalized_1은 100 기준이므로 맞춤
else:
    print("NASDAQ 데이터가 없습니다")
    rs_df = None

if rs_df is not None:
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.set_yscale('log')
    
    # 섹터별로 개별 종목 RS 그리기
    for sector, tickers in sector_df.groupby(sector_df):
        color = sector_colors.get(sector, 'grey')
        for ticker in tickers.index:
            if ticker in rs_df.columns and ticker != '^IXIC':
                ax.plot(rs_df[ticker], color=color, alpha=0.25, linewidth=0.6)
        # 범례용 더미 라인
        count = len([t for t in tickers.index if t in rs_df.columns and t != '^IXIC'])
        if count > 0:
            ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({count})')
    
    # 전체 RS 중앙값
    rs_median = rs_df.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    ax.plot(rs_median, color='black', linewidth=2.5, linestyle='--', label='Median RS')
    
    # 기준선: RS=1은 나스닥과 동일 수익률
    ax.axhline(1, color='red', linestyle='-', linewidth=2, label='RS=1 (NASDAQ)')
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Relative Strength vs NASDAQ (Log)')
    ax.set_title('Relative Strength by Sector')
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    plt.tight_layout()
    plt.show()

In [ ]:
# 기간별 RS 차트 (4개 구간으로 분할)
if rs_df is not None:
    # 기간 정의
    periods = [
        ('1988.10 ~ 1991.12', '1988-10-01', '1991-12-31'),
        ('1992.01 ~ 1994.12', '1992-01-01', '1994-12-31'),
        ('1995.01 ~ 1997.12', '1995-01-01', '1997-12-31'),
        ('1998.01 ~ 2000.03', '1998-01-01', '2000-03-31'),
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, (period_name, start, end) in enumerate(periods):
        ax = axes[idx]
        ax.set_yscale('log')
        
        # 해당 기간 데이터 슬라이싱
        rs_period = rs_df.loc[start:end]
        
        # 기간 시작점 기준으로 다시 정규화 (각 기간의 시작=1)
        rs_period_norm = rs_period / rs_period.iloc[0]
        
        # 섹터별로 개별 종목 RS 그리기
        for sector, tickers in sector_df.groupby(sector_df):
            color = sector_colors.get(sector, 'grey')
            for ticker in tickers.index:
                if ticker in rs_period_norm.columns and ticker != '^IXIC':
                    ax.plot(rs_period_norm[ticker], color=color, alpha=0.5, linewidth=0.6)
        
        # 해당 기간 RS 중앙값
        rs_median = rs_period_norm.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
        ax.plot(rs_median, color='black', linewidth=2.5, linestyle='--', label='Median RS')
        
        # 기준선
        ax.axhline(1, color='red', linestyle='-', linewidth=1.5, label='RS=1 (NASDAQ)')
        
        ax.set_title(f'{period_name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('RS (period start=1)')
        ax.legend(fontsize=8, loc='upper left')
    
    plt.suptitle('Relative Strength by Period', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
def classify_rs_pattern(rs_series, median_series, threshold=0.1):
    """
    RS 패턴을 4가지로 분류
    1: 항상 위 (전 기간 중앙값/1 위)
    2: 골든크로스 (약하다가 치고 오름)
    3: 데드크로스 (강하다가 치고 내려감)
    4: 항상 아래 (전 기간 중앙값/1 아래)
    """
    rs = rs_series.dropna()
    med = median_series.loc[rs.index]
    
    if len(rs) < 10:
        return None
    
    # 기간을 전반/후반으로 나눔
    mid = len(rs) // 2
    first_half = rs.iloc[:mid]
    second_half = rs.iloc[mid:]
    med_first = med.iloc[:mid]
    med_second = med.iloc[mid:]
    
    # 전반/후반 평균이 중앙값 대비 위인지 아래인지
    first_above = (first_half > med_first).mean() > 0.6
    first_below = (first_half < med_first).mean() > 0.6
    second_above = (second_half > med_second).mean() > 0.6
    second_below = (second_half < med_second).mean() > 0.6
    
    if first_above and second_above:
        return 1  # 항상 위
    elif first_below and second_above:
        return 2  # 골든크로스
    elif first_above and second_below:
        return 3  # 데드크로스
    elif first_below and second_below:
        return 4  # 항상 아래
    else:
        return 0  # 분류 불가 (혼재)

# 기간별로 분류 실행
periods = [
    ('1988.10~1991.12', '1988-10-01', '1991-12-31'),
    ('1992.01~1994.12', '1992-01-01', '1994-12-31'),
    ('1995.01~1997.12', '1995-01-01', '1997-12-31'),
    ('1998.01~2000.03', '1998-01-01', '2000-03-31'),
]

pattern_results = {}

for period_name, start, end in periods:
    rs_period = rs_df.loc[start:end]
    rs_period_norm = rs_period / rs_period.iloc[0]
    rs_median = rs_period_norm.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    
    period_patterns = {}
    for ticker in rs_period_norm.columns:
        if ticker == '^IXIC':
            continue
        pattern = classify_rs_pattern(rs_period_norm[ticker], rs_median)
        if pattern is not None:
            period_patterns[ticker] = pattern
    
    pattern_results[period_name] = period_patterns
    
    # 결과 요약
    from collections import Counter
    counts = Counter(period_patterns.values())
    print(f"\n=== {period_name} ===")
    print(f"1. 항상 위: {counts.get(1, 0)}개")
    print(f"2. 골든크로스: {counts.get(2, 0)}개")
    print(f"3. 데드크로스: {counts.get(3, 0)}개")
    print(f"4. 항상 아래: {counts.get(4, 0)}개")
    print(f"0. 분류불가: {counts.get(0, 0)}개")

In [ ]:
# 전 기간 걸쳐 일관된 패턴을 보이는 종목 찾기
from collections import defaultdict

# 종목별 기간별 패턴 취합
ticker_patterns = defaultdict(list)
for period_name, patterns in pattern_results.items():
    for ticker, pattern in patterns.items():
        ticker_patterns[ticker].append((period_name, pattern))

# 항상 위 (4개 기간 모두 1번)
always_above = [t for t, patterns in ticker_patterns.items() 
                if len(patterns) == 4 and all(p[1] == 1 for p in patterns)]

# 항상 아래 (4개 기간 모두 4번)
always_below = [t for t, patterns in ticker_patterns.items() 
                if len(patterns) == 4 and all(p[1] == 4 for p in patterns)]

# 상승 추세 (초기에 아래→후기에 위)
rising = [t for t, patterns in ticker_patterns.items()
          if len(patterns) == 4 and patterns[0][1] in [4, 0] and patterns[3][1] == 1]

# 하락 추세 (초기에 위→후기에 아래)
falling = [t for t, patterns in ticker_patterns.items()
           if len(patterns) == 4 and patterns[0][1] == 1 and patterns[3][1] in [4, 0]]

print("=== 전 기간 일관된 패턴 ===\n")

print(f"1. 12년간 항상 나스닥을 이긴 종목 ({len(always_above)}개):")
for t in always_above:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n2. 12년간 항상 나스닥에 진 종목 ({len(always_below)}개):")
for t in always_below:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n3. 초기 약세 → 후기 강세로 전환 ({len(rising)}개):")
for t in rising:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n4. 초기 강세 → 후기 약세로 전환 ({len(falling)}개):")
for t in falling:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

### RS 분석 결과

**12년간 일관되게 나스닥을 이긴 종목**: AB, AEG (Financial Services), LEG (Consumer Cyclical) — 단 3개

**초기 약세 → 후기 강세 전환 (25개)**: AAPL, AMAT, BBY, COST, SPGI 등

**초기 강세 → 후기 약세 전환 (24개)**: BA, NSC, BEN, SLB 등 전통 산업재/에너지

---

### 관찰된 패턴

1. **12년간 일관된 초과수익 종목은 극소수** — 162개 중 3개(1.9%)만이 전 기간 나스닥을 상회. 대부분의 종목은 특정 시기에만 강세를 보임

2. **후반부 강세 전환 종목의 특징** — AAPL, AMAT, BBY 등은 초기 7~8년간 지수 대비 약세였다가 후반부에 급등. 이 패턴이 "캐즘 후 성장"인지 "우연"인지는 추가 검증 필요

3. **전통 산업재의 상대적 약세** — BA, NSC, SLB 등 설비 집약적 종목들은 초기 강세 후 후반부에 지수 대비 약세로 전환

4. **Financial Services의 안정적 성과** — 일관된 초과수익 3개 종목 중 2개가 금융 섹터. 다만 이것이 "기술 혁신의 간접 수혜" 때문인지는 데이터만으로 단정할 수 없음

## 7. 그룹 2 분석 — 버블 말기 1년 (1999.03 ~ 2000.03)

339개 종목 전체를 대상으로 버블 말기 1년간의 패턴을 분석한다. 12년 전체를 경험한 그룹 1과 비교하여 **버블 말기에 새로 진입한 종목들의 특성**을 파악한다.

In [ ]:
# 그룹 2 정규화 (1999.03 시작점 = 100)
normalized_2 = close_df_2.apply(lambda x: x / x.iloc[0] * 100)

# 그룹 2 섹터 정보 수집 (그룹 1에 없는 종목만 추가)
sectors_2 = {}
new_tickers = [t for t in close_df_2.columns if t not in sector_df.index]
print(f"그룹 2 신규 종목: {len(new_tickers)}개")

for ticker in new_tickers:
    try:
        info = yf.Ticker(ticker).info
        sectors_2[ticker] = info.get('sector', 'Unknown')
    except:
        sectors_2[ticker] = 'Unknown'

# 기존 섹터 정보와 병합
sector_df_2 = pd.concat([sector_df, pd.Series(sectors_2)])
sector_df_2 = sector_df_2[~sector_df_2.index.duplicated(keep='first')]

print(f"그룹 2 섹터 분포:")
print(sector_df_2[close_df_2.columns].value_counts())

In [ ]:
# 그룹 2 섹터별 로그 차트
plot_sector_chart(normalized_2, sector_df_2[close_df_2.columns], sector_colors, 
                  '1999.03 ~ 2000.03 Rally by Sector (339 stocks)')

In [ ]:
# 그룹 2 클러스터링 (k=4)
kmeans_2, cluster_df_2, X_2 = run_clustering(normalized_2, n_clusters=4)
plot_cluster_chart(normalized_2, cluster_df_2, sector_df_2, sector_colors, n_clusters=4)

In [ ]:
# 그룹 2 클러스터별 통계 요약
summarize_clusters(normalized_2, cluster_df_2, sector_df_2, n_clusters=4)

### 그룹 2 분석 결과

| 클러스터 | 종목 수 | 1년 수익률 (중앙값) | 주요 섹터 |
|----------|---------|---------------------|-----------|
| 폭등 | ~30개 | ~4배 | Technology 다수 |
| 상승 | ~100개 | ~1.5배 | 전 섹터 분포 |
| 횡보 | ~100개 | ~1배 | 전 섹터 분포 |
| 하락 | ~100개 | ~-50% | Financial Services, Industrials |

**그룹 2 특징**:
- 1년이라는 짧은 기간에도 **극단적 양극화** — 4배 상승과 -50% 하락이 공존
- **Technology 섹터가 폭등 클러스터에 집중** — 12년 분석과 달리 버블 말기에는 테크 직접 수혜
- **손실 비율이 높음** — 339개 중 약 37%가 마이너스 수익률

## 8. 결론

### 그룹 1 vs 그룹 2 비교

| 지표 | 그룹 1 (12년) | 그룹 2 (1년) |
|------|---------------|--------------|
| 중앙값 수익률 | 3.88배 | 1.08배 |
| 95% 신뢰구간 | **[3.08배, 4.35배]** | **[1.02배, 1.12배]** |
| 손실 종목 비율 | 2% (4개) | 37% (~125개) |
| 슈퍼 성장 주도 섹터 | Financial Services | Technology |

### 통계적 검증 결과

| 검증 | 그룹 1 | 그룹 2 | 해석 |
|------|--------|--------|------|
| Kruskal-Wallis | 유의미 | 유의미 | 클러스터 구분이 통계적으로 의미있음 |
| 카이제곱 | **독립** | **연관** | 그룹1은 섹터 무관, 그룹2는 Tech 집중 |
| Bootstrap 95% CI | [3.08x, 4.35x] | [1.02x, 1.12x] | 두 그룹 CI 겹치지 않음 → 차이 유의미 |

### 핵심 발견

**그룹 1 (12년 장기)**:
- 섹터와 성과가 독립적 → **섹터보다 종목 선별이 중요**
- Bootstrap 하한 3.08배 → 최악의 경우에도 3배 이상 (Survivorship Bias 감안 필요)
- 슈퍼 성장 클러스터에서 Financial Services(14개) > Technology(7개)

**그룹 2 (버블 말기 1년)**:
- 섹터와 성과가 연관됨 → **Tech/바이오가 성장 클러스터에 집중**
- 중앙값 1.08배, CI도 [1.02x, 1.12x]로 좁음 → 대부분 본전치기
- 37% 손실 vs 일부 4배 → **"운 좋으면 4배, 아니면 -50%" 도박장 구조**

### 한계

- **Survivorship Bias**: 상장폐지 종목 누락. 손실 비율(2%, 37%)은 과소추정
- "12년 보유 시 최소 3배"는 **생존한 종목에 한정된 결과**
- 추후 FMP API로 상장폐지 종목 포함 재분석 필요

### 후속 과제

- [ ] FMP API로 Survivorship Bias 해소
- [ ] 2009~2021 빅테크 랠리 분석
- [ ] 현재 AI 랠리의 RS 분석으로 초기/중기/말기 판단